# Questão 3

Usando algoritmos implementados na questão 2, aproxime as funções abaixo. Compare os
resultados com as curvas exatas, para o caso dos itens b e c e apresente para cada caso a
curva do erro médio de treinamento com relação ao número de épocas e a curva do erro médio com o conjunto de validação. Faça uma análise comparativa sobre a convergência de cada um dos algoritmos.

    a) a função lógica XOR;
    b) f(x)= sin(pi*x)/(pi*x), 0 <= x <= 4

# Conteúdo da questão anterior

## Importações utilizadas

In [89]:
from numpy import exp, array, random, dot, shape, zeros, transpose, matrix, asscalar
from math import ceil, sinh, cosh, floor
import matplotlib.pyplot as plt
#from sklearn.metrics import classification_report
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import accuracy_score

## Classes desenvolvidas

In [90]:
class NeuronLayer():
    """
    Classe representando a camada de neurônios da rede
    Parâmetros:
        - num_neurons: o número de neurônios na camada
        - num_inputs: o número de entradas que cada neurônio da camada terá
    """
    def __init__(self, num_neurons, num_inputs):
        self.weights = 2 * random.random((num_inputs, num_neurons)) - 1
        
class NeuralNetwork():
    def __init__(self, layer1, layer2):
        self.layer1 = layer1
        self.layer2 = layer2

    def activation_func(self, func_name, z):
        """
        Executa uma das funções de ativação da rede
        Parâmetros:
            - func_name: nome da função de ativação
            - z: valor a ser aplicado na função de ativação
        """
        if(func_name == 'sigmoid'):
            return 1 / (1 + exp(-z))
        elif(func_name == 'tanh'):
            return sinh(z) / cosh(z)
        elif(func_name == 'relu'):
            if(z <= 0):
                return 0
            else:
                return z
        elif(func_name == 'sigmoid_deriv'):
            return z * (1-z)
        elif(func_name == 'relu_deriv'):
            if(z<=0):
                return 0
            else:
                return 1

    def train_network(self, inputs, expected_outs, num_iterations, learning_rate, backprop_type):
        """
        Método para treinamento da rede neural e ajuste dos pesos sinápticos
        Parâmetros:
            - inputs: o conjunto de treinamento
            - expected_outs: saídas desejadas do conjunto de treinamento
            - num_iterations: épocas (duração do treinamento)
            - learning_rate: taxa de aprendizado
            - backprop_type: o tipo de retropropagação a ser usado no treinamento ("momentum", "batch", "stochastic)
            
        """
        alpha = 0.1                                     #constante do momento (0 <= alpha < 1)
        samples, features = shape(inputs)
        layer1_adjustment = 0
        layer2_adjustment = 0
        
        for count in range(num_iterations):
            #print("época " + str(count + 1) + ":")

            if((backprop_type == "batch") or (backprop_type == "momentum")):
                out_l1, out_l2 = self.classify(inputs)
                
                ### Cálculo dos erros na camada 2
                layer2_error = expected_outs - out_l2
                layer2_grad = layer2_error * self.activation_func("sigmoid_deriv",out_l2)      # gradiente local do neurônio da camada de saída
                #print("erro camada 2: \n" + str(layer2_error))

                ## Com base no resultado da camada 2, calcular o erro na camada 1
                layer1_error = layer2_grad.dot(self.layer2.weights.T)
                layer1_grad = layer1_error * self.activation_func("sigmoid_deriv",out_l1)                 # gradiente da camada 1
                #print("erro camada 1: \n" + str(layer1_error))
                
                if(backprop_type == "momentum"):
                    if(count == 0):
                        layer1_adjustment = (inputs.T.dot(layer1_grad)) * learning_rate
                        layer2_adjustment = out_l1.T.dot(layer2_grad) * learning_rate
                    else:
                        ## Cálculo da regra delta com constante do momento
                        layer1_adjustment = (alpha * temp1) + ((inputs.T.dot(layer1_grad)) * learning_rate)
                        layer2_adjustment = (alpha * temp2) +(out_l1.T.dot(layer2_grad) * learning_rate)
                else:
                    ## Cálculo normal da regra delta
                    layer1_adjustment = (inputs.T.dot(layer1_grad)) * learning_rate
                    layer2_adjustment = out_l1.T.dot(layer2_grad) * learning_rate

                self.layer1.weights += layer1_adjustment
                temp1 = self.layer1.weights
                self.layer2.weights += layer2_adjustment
                temp2 = self.layer2.weights
            else:
                #treinamento estocástico
                for count2 in range(samples):
                    out_l1, out_l2 = self.classify(inputs[count2, :])
                    #print("saída camada 1: \n" + str(out_l1) + "\nsaída camada 2: \n" + str(out_l2))

                    ### Cálculo dos erros na camada 2
                    layer2_error = expected_outs[count2, :] - out_l2
                    layer2_grad = layer2_error * self.activation_func("sigmoid_deriv",out_l2)                 # gradiente local do neurônio da camada de saída

                    ## Com base no resultado da camada 2, calcular o erro na camada 1
                    layer1_error = layer2_grad.dot(self.layer2.weights.T)
                    layer1_grad = layer1_error * self.activation_func("sigmoid_deriv",out_l1)                 # gradiente da camada 1

                    ## Regra delta
                    ar = array([inputs[count2, :]])
                    l1_grad = array([layer1_grad])
                    o_l1 = array([out_l1])
                    l2_grad = array([out_l2])
                    
                    layer1_adjustment = (ar.T.dot(l1_grad)) * learning_rate
                    layer2_adjustment = o_l1.T.dot(l2_grad) * learning_rate

                    self.layer1.weights += layer1_adjustment
                    temp1 = self.layer1.weights
                    self.layer2.weights += layer2_adjustment
                    temp2 = self.layer2.weights

    def classify(self, inputs):
        """
        Classifica uma entrada pela rede neural
        Parâmetros:
            - inputs: valor de um conjunto de treinamento/classificação
        """
        output_from_layer1 = self.activation_func("sigmoid", (dot(inputs, self.layer1.weights)))
        output_from_layer2 = self.activation_func("sigmoid", (dot(output_from_layer1, self.layer2.weights)))
        return output_from_layer1, output_from_layer2

    def print_weights(self):
        print("Pesos da camada 1: ")
        print (self.layer1.weights)
        print("Pesos da camada 2: ")
        print (self.layer2.weights)

## Informações adicionais

Foi mantida a implementação de um perceptron com apenas duas camadas. Para a questão 3, mantém-se as duas camadas de neurônios, sendo a primeira com 5 neurônios, recebendo as entradas do conjunto de treinamento, e a segunda contendo apenas 1 neurônio, recebendo como entrada as saídas dos neurônios da camada anterior.

O conjunto de treinamento/classificação da questão 3a contém 200 entradas.

In [96]:
if __name__ == "__main__":
    class_errados = 0
    x_3a = []
    y_3a = []
    for line in open('./input_q3a_test.txt', 'r').readlines():
        x_3a.append([float(num) for num in line.split(',')])

    for line in open('./output_q3a_test.txt', 'r').readlines():
        y_3a.append([int(line)])

    x_3a = array(x_3a)
    y_3a = array(y_3a)
    smp, feat = shape(x_3a)
    
    LEARNING_RATE = 0.3

    random.seed(1)

    # Rede neural de duas camadas (setando valores (numero de neurônios e seu numero de entradas))
    layer1 = NeuronLayer(5, 2)
    layer2 = NeuronLayer(1, 5)

    neural_network = NeuralNetwork(layer1, layer2)

    # Atribuição randomica de pesos a rede neural
    print ("Pesos antes de iniciar o treinamento: ")
    neural_network.print_weights()

    # Conjunto de treinamento (2 exemplos com 2 valores de entrada e 1 valor de saída).
    training_set_inputs = array([[0, 1], [0, 0], [1, 0], [1, 1]])
    training_set_outputs = array([[1, 0, 1, 0]]).T

    # treinamento da rede neural
    neural_network.train_network(x_3a, y_3a, 300, LEARNING_RATE, "momentum")

    print ("Pesos de saída (após última iteração do treinamento): ")
    neural_network.print_weights()

    print("Erros obtidos com relação aos conjuntos treinados: ")
    for cl_count in range(smp):
        #print(str(x_t[cl_count,:]) + ':' + str(classify(x_t[cl_count,:])))
        h, out = neural_network.classify(x_3a[cl_count,:])
        if((asscalar(out) - floor(asscalar(out))) > 0.5):
            out = ceil(asscalar(out))
        else:
            out = floor(asscalar(out))
        
        if(out != y_3a[cl_count,:]):
            print(str(cl_count) + "/" + str(out) + "\t|\t" + str(y_3a[cl_count,:]))
            class_errados = class_errados + 1

    print("erros: " + str(class_errados) + "/" + str(smp))

    #print ("Teste com entradas não treinadas: ")
    #h, output = neural_network.classify(array([1, 1]))
    #print("[1, 1] ==>" + str(output))
    #h, out2 = neural_network.classify(array([1, 0]))
    #print ("[1, 0] ==> " + str(out2))

Pesos antes de iniciar o treinamento: 
Pesos da camada 1: 
[[-0.16595599  0.44064899 -0.99977125 -0.39533485 -0.70648822]
 [-0.81532281 -0.62747958 -0.30887855 -0.20646505  0.07763347]]
Pesos da camada 2: 
[[-0.16161097]
 [ 0.370439  ]
 [-0.5910955 ]
 [ 0.75623487]
 [-0.94522481]]
Pesos de saída (após última iteração do treinamento): 
Pesos da camada 1: 
[[-1.73999133e+12  6.52907755e+11 -4.13268359e+12 -2.81691420e+12
  -3.13344852e+12]
 [-2.62198223e+12 -3.65477891e+12 -3.77386437e+12  2.08403317e+11
  -2.78570855e+12]]
Pesos da camada 2: 
[[-9.00904556e+11]
 [ 2.13321216e+12]
 [-4.11406961e+12]
 [ 2.24851561e+12]
 [-2.63301294e+12]]
Erros obtidos com relação aos conjuntos treinados: 
erros: 0/200


# Testes de saídas

## Questão 3a

## Regra delta com termo do momento (alpha = 0.1)

#### Parâmetros:
    - Neurônios (camada 1): 2
    - Neurônios (camada 2): 1
    - Taxa de aprendizado: 0.03 (teste 1), 0.3 (teste 2)
    - Épocas: 7000 (teste 1), 300 (teste 2)

##### Pesos antes de iniciar o treinamento:

Pesos da camada 1:

    [[-0.16595599  0.44064899]
     [-0.99977125 -0.39533485]]
 
Pesos da camada 2: 

    [[-0.70648822]
     [-0.81532281]]
 
##### Pesos de saída (após última iteração do treinamento): 

Pesos da camada 1: 

    [[-2.27292820e+289  1.61165308e+289]
     [-5.42359830e+289 -2.32909492e+289]]
   
Pesos da camada 2: 

    [[-1.77661396e+289]
     [ 4.62099080e+288]]
 
Erros obtidos com os conjuntos treinados/classificados: 

    1/0 [1]
    5/0 [1]
    10/0 [1]
    12/0 [1]
    16/0 [1]
    33/0 [1]
    40/0 [1]
    43/0 [1]
    45/0 [1]
    48/0 [1]
    52/0 [1]
    55/0 [1]
    56/0 [1]
    57/0 [1]
    60/0 [1]
    70/0 [1]
    72/0 [1]
    74/0 [1]
    76/0 [1]
    77/0 [1]
    80/0 [1]
    81/0 [1]
    84/0 [1]
    96/0 [1]
    104/0 [1]
    105/0 [1]
    108/0 [1]
    115/0 [1]
    120/0 [1]
    134/0 [1]
    137/0 [1]
    138/0 [1]
    139/0 [1]
    145/0 [1]
    148/0 [1]
    157/0 [1]
    160/0 [1]
    164/0 [1]
    170/0 [1]
    172/0 [1]
    181/0 [1]
    187/0 [1]
    190/0 [1]
    196/0 [1]
    197/0 [1]

#### erros: 45/200

### 3º Teste

#### Parâmetros:
    - Neurônios (camada 1): 5
    - Neurônios (camada 2): 1
    - Taxa de aprendizado: 0.3
    - Épocas: 300

##### Pesos antes de iniciar o treinamento:

Pesos da camada 1:

    [[-0.16595599  0.44064899 -0.99977125 -0.39533485 -0.70648822]
     [-0.81532281 -0.62747958 -0.30887855 -0.20646505  0.07763347]]
 
Pesos da camada 2: 

    [[-0.16161097]
     [ 0.370439  ]
     [-0.5910955 ]
     [ 0.75623487]
     [-0.94522481]]

##### Pesos de saída (após última iteração do treinamento): 

Pesos da camada 1: 

    [[-1.73999133e+12  6.52907755e+11 -4.13268359e+12 -2.81691420e+12
      -3.13344852e+12]
     [-2.62198223e+12 -3.65477891e+12 -3.77386437e+12  2.08403317e+11
      -2.78570855e+12]]
   
Pesos da camada 2: 

    [[-9.00904556e+11]
     [ 2.13321216e+12]
     [-4.11406961e+12]
     [ 2.24851561e+12]
     [-2.63301294e+12]]
 
Erros obtidos com os conjuntos treinados/classificados:
#### erros: 0/200

## ======================================================================   
    
## Backpropagation por lotes

#### Parâmetros:
    - Neurônios (camada 1): 2
    - Neurônios (camada 2): 1
    - Taxa de aprendizado: 0.03
    - Épocas: 7000 (teste 1), 300 (teste 2)

##### Pesos antes de iniciar o treinamento:

Pesos da camada 1:

    [[-0.16595599  0.44064899]
     [-0.99977125 -0.39533485]]
 
Pesos da camada 2: 

    [[-0.70648822]
     [-0.81532281]]
 
##### Pesos de saída (após última iteração do treinamento): 

Pesos da camada 1: 

    [[-5.34716684  1.61636268]
     [-9.47389516 -9.78770329]]
   
Pesos da camada 2: 

    [[-13.00886677]
     [  4.99981404]]
 
Erros obtidos com os conjuntos treinados/classificados:

    1/0 [1]
    5/0 [1]
    9/1 [0]
    10/0 [1]
    11/1 [0]
    12/0 [1]
    16/0 [1]
    25/1 [0]
    28/1 [0]
    33/0 [1]
    38/1 [0]
    40/0 [1]
    43/0 [1]
    44/1 [0]
    45/0 [1]
    46/1 [0]
    48/0 [1]
    49/1 [0]
    51/1 [0]
    52/0 [1]
    55/0 [1]
    56/0 [1]
    57/0 [1]
    60/0 [1]
    63/1 [0]
    65/1 [0]
    70/0 [1]
    72/0 [1]
    74/0 [1]
    76/0 [1]
    77/0 [1]
    79/1 [0]
    80/0 [1]
    81/0 [1]
    84/0 [1]
    87/1 [0]
    88/1 [0]
    91/1 [0]
    92/1 [0]
    96/0 [1]
    98/1 [0]
    99/1 [0]
    101/1 [0]
    102/1 [0]
    104/0 [1]
    105/0 [1]
    107/1 [0]
    108/0 [1]
    110/1 [0]
    112/1 [0]
    115/0 [1]
    117/1 [0]
    120/0 [1]
    122/1 [0]
    124/1 [0]
    125/1 [0]
    130/1 [0]
    132/1 [0]
    134/0 [1]
    137/0 [1]
    138/0 [1]
    139/0 [1]
    140/1 [0]
    144/1 [0]
    145/0 [1]
    146/1 [0]
    147/1 [0]
    148/0 [1]
    149/1 [0]
    152/1 [0]
    153/1 [0]
    157/0 [1]
    160/0 [1]
    162/1 [0]
    164/0 [1]
    166/1 [0]
    167/1 [0]
    168/1 [0]
    170/0 [1]
    172/0 [1]
    173/1 [0]
    177/1 [0]
    178/1 [0]
    179/1 [0]
    181/0 [1]
    184/1 [0]
    186/1 [0]
    187/0 [1]
    190/0 [1]
    191/1 [0]
    193/1 [0]
    195/1 [0]
    196/0 [1]
    197/0 [1]
    
#### erros: 94/200

### 3º Teste

#### Parâmetros:
    - Neurônios (camada 1): 5
    - Neurônios (camada 2): 1
    - Taxa de aprendizado: 0.03
    - Épocas: 300

##### Pesos antes de iniciar o treinamento:

Pesos da camada 1:

    [[-0.16595599  0.44064899 -0.99977125 -0.39533485 -0.70648822]
     [-0.81532281 -0.62747958 -0.30887855 -0.20646505  0.07763347]]
 
Pesos da camada 2: 

    [[-0.16161097]
     [ 0.370439  ]
     [-0.5910955 ]
     [ 0.75623487]
     [-0.94522481]]

##### Pesos de saída (após última iteração do treinamento): 

Pesos da camada 1: 

    [[-0.60954743  3.67807678 -5.7222896  -6.73062165  2.81402682]
     [-1.14172501 -7.58120461 -5.6219779   3.08148804 -1.13208129]]
   
Pesos da camada 2: 

    [[ -1.05126187]
     [  9.52382592]
     [-17.20530964]
     [  5.93749338]
     [ -5.10104222]]
 
Erros obtidos com os conjuntos treinados/classificados:
#### erros: 0/200

## ======================================================================   
    
### Backpropagation estocástico

#### Parâmetros:
    - Neurônios (camada 1): 5
    - Neurônios (camada 2): 1
    - Taxa de aprendizado: 0.3
    - Épocas: 300

##### Pesos antes de iniciar o treinamento:

Pesos da camada 1:

    [[-0.16595599  0.44064899 -0.99977125 -0.39533485 -0.70648822]
     [-0.81532281 -0.62747958 -0.30887855 -0.20646505  0.07763347]]
 
Pesos da camada 2: 

    [[-0.16161097]
     [ 0.370439  ]
     [-0.5910955 ]
     [ 0.75623487]
     [-0.94522481]]

##### Pesos de saída (após última iteração do treinamento): 

Pesos da camada 1: 

    [[-0.17701543  0.41801125 -1.00585525 -0.41761916 -0.7124805 ]
     [-0.82551895 -0.64341212 -0.31913473 -0.2170345   0.06361007]]
   
Pesos da camada 2: 

    [[6968.20131037]
     [8632.26813296]
     [6250.03425577]
     [7594.68077771]
     [7525.28891091]]

Erros obtidos com os conjuntos treinados/classificados:

    0/1	|	[0]
    2/1	|	[0]
    4/1	|	[0]
    6/1	|	[0]
    7/1	|	[0]
    9/1	|	[0]
    11/1	|	[0]
    13/1	|	[0]
    18/1	|	[0]
    19/1	|	[0]
    20/1	|	[0]
    22/1	|	[0]
    23/1	|	[0]
    25/1	|	[0]
    28/1	|	[0]
    30/1	|	[0]
    34/1	|	[0]
    36/1	|	[0]
    37/1	|	[0]
    38/1	|	[0]
    44/1	|	[0]
    46/1	|	[0]
    49/1	|	[0]
    50/1	|	[0]
    51/1	|	[0]
    54/1	|	[0]
    58/1	|	[0]
    59/1	|	[0]
    62/1	|	[0]
    63/1	|	[0]
    65/1	|	[0]
    67/1	|	[0]
    68/1	|	[0]
    71/1	|	[0]
    73/1	|	[0]
    78/1	|	[0]
    79/1	|	[0]
    82/1	|	[0]
    83/1	|	[0]
    87/1	|	[0]
    88/1	|	[0]
    90/1	|	[0]
    91/1	|	[0]
    92/1	|	[0]
    94/1	|	[0]
    97/1	|	[0]
    98/1	|	[0]
    99/1	|	[0]
    100/1	|	[0]
    101/1	|	[0]
    102/1	|	[0]
    106/1	|	[0]
    107/1	|	[0]
    110/1	|	[0]
    111/1	|	[0]
    112/1	|	[0]
    116/1	|	[0]
    117/1	|	[0]
    121/1	|	[0]
    122/1	|	[0]
    124/1	|	[0]
    125/1	|	[0]
    126/1	|	[0]
    129/1	|	[0]
    130/1	|	[0]
    132/1	|	[0]
    133/1	|	[0]
    135/1	|	[0]
    136/1	|	[0]
    140/1	|	[0]
    141/1	|	[0]
    144/1	|	[0]
    146/1	|	[0]
    147/1	|	[0]
    149/1	|	[0]
    150/1	|	[0]
    152/1	|	[0]
    153/1	|	[0]
    156/1	|	[0]
    158/1	|	[0]
    161/1	|	[0]
    162/1	|	[0]
    165/1	|	[0]
    166/1	|	[0]
    167/1	|	[0]
    168/1	|	[0]
    173/1	|	[0]
    176/1	|	[0]
    177/1	|	[0]
    178/1	|	[0]
    179/1	|	[0]
    180/1	|	[0]
    184/1	|	[0]
    185/1	|	[0]
    186/1	|	[0]
    188/1	|	[0]
    191/1	|	[0]
    193/1	|	[0]
    195/1	|	[0]
    198/1	|	[0]

#### erros: 100/200

## Questão 3b